In [1]:
import json
import random
from pathlib import Path

import pandas as pd
import spacy
import textacy
from tqdm.notebook import tqdm
from spacy.lang.en import English

%load_ext autoreload
%autoreload 2
Path.ls = lambda x: list(x.iterdir())

In [2]:
data_dir = Path("../data").resolve()
assert data_dir.exists()
data_dir.ls()

[PosixPath('/home/nirant/AppReview/data/Clubhouse_us_app_store_reviews.json'),
 PosixPath('/home/nirant/AppReview/data/frequency_count.json'),
 PosixPath('/home/nirant/AppReview/data/Moj_us_app_store_reviews.json'),
 PosixPath('/home/nirant/AppReview/data/Uber_us_app_store_reviews.json'),
 PosixPath('/home/nirant/AppReview/data/com.ubercab_us_play_store_reviews.json'),
 PosixPath('/home/nirant/AppReview/data/Netflix_us_app_store_reviews.json'),
 PosixPath('/home/nirant/AppReview/data/Moj_in_app_store_reviews.json'),
 PosixPath('/home/nirant/AppReview/data/IndiaGold_in_app_store_reviews.json')]

Note:

Here we will explore App Reviews for just one app: Uber (Passenger/Cab, not the Driver). The additional data to reproduce this for other clients is left as an exercise for you.

But to get an overview of all of them, we combine them into a larger single text string and explore them. 

In [3]:
file_path = data_dir / "Uber_us_app_store_reviews.json"; assert file_path.exists()
with file_path.open("r") as f:
    raw_data = pd.read_json(f)
    reviews = " ".join(raw_data["review"].to_list())
# print(type(reviews), reviews)

In [4]:
reviews = []
files = [x for x in data_dir.ls() if "app" in x.name]
for file_path in files:
    try:
        with file_path.open("r") as f:
            file_data = pd.read_json(f)
            reviews += file_data["review"].to_list()
    except ValueError as e:
        print(f"Value Error with {file.name}")
    except KeyError as ke:
        print(f"Key error with {file.name}")

In [13]:
# !python -m spacy download en_core_web_sm

In [14]:
nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("en_core_web_trf")

In [6]:
%time reviews = [rev.strip() for rev in reviews]
len(reviews)

CPU times: user 698 µs, sys: 465 µs, total: 1.16 ms
Wall time: 1.18 ms


6049

In [46]:
%time corpus = textacy.Corpus("en_core_web_sm", data=reviews)

CPU times: user 51.4 s, sys: 1.85 s, total: 53.2 s
Wall time: 53.6 s


In [47]:
corpus.n_docs, corpus.n_sents, corpus.n_tokens

(6049, 35560, 592458)

In [59]:
word_counts = corpus.word_counts(by="lemma_", filter_stops= True, filter_nums=True, filter_punct=True)
sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:25]

[('app', 4563),
 ('Netflix', 3396),
 ('watch', 3072),
 ('like', 2768),
 ('show', 2420),
 ('movie', 2409),
 ('driver', 2374),
 ('Uber', 2337),
 ('time', 2283),
 ('love', 1814),
 ('people', 1382),
 ('ride', 1369),
 ('good', 1361),
 ('want', 1352),
 ('know', 1257),
 ('go', 1135),
 ('try', 1117),
 ('great', 1098),
 ('use', 1094),
 ('need', 1077),
 ('thing', 1067),
 ('say', 1020),
 ('get', 1019),
 ('work', 1014),
 ('way', 989)]

Since we've limited memory on this machine, disabling memory heavy parts of the pipeline. If we combine all reviews into one large chunk -- this is what we get. 

```python-traceback
ValueError: [E088] Text of length 2737687 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`
 ```

In [7]:
# nlp = spacy.load("en_core_web_trf", disable=["parser", "ner"])
# nlp.max_length = len(reviews) + 1
# nlp(reviews)

Even after disabling the components, it will take extremely long durations and a lot of memory to process our relatively "small data". The solution? Batch your data

In [8]:
nlp = English()
# Create a Tokenizer with the default settings for English
# including punctuation rules and exceptions
tokenizer = nlp.tokenizer

In [ ]:
from collections import Counter
from typing import List


class TextSummary:
    def __init__(self, records: List[str]):
        self.token_summary = self.make_summary(self.extract_tokens(records))
        self.token_stats = self.get_corpora_stats(self.token_summary)

    def extract_tokens(self, records: List[str]):
        token_vocab = []
        for doc in tokenizer.pipe(records):
            token_vocab.append(Counter([str(x) for x in doc]))
        return token_vocab

    def make_summary(self, record_summaries: List[Counter]):
        """Get a Count Distribution for the entire Corpora"""
        corpora_summary = record_summaries[0]
        for record in tqdm(record_summaries[1:]):
            corpora_summary += record

        return corpora_summary

    def get_corpora_stats(self, summary: List[Counter]):
        self.vocab = list(summary.keys())
        self.vocab_sz = len(self.vocab)
        self.size = sum(summary[key] for key in summary.keys())
        return {"size": self.size, "vocab_sz": self.vocab_sz, "vocab": self.vocab}

In [ ]:
tsm = TextSummary(reviews)

In [ ]:
tsm.token_summary.most_common(25)

In [44]:
%%time
for doc in tqdm(nlp.pipe(reviews, batch_size=100, n_process=-1), total=len(reviews)):
    for chunk in doc.noun_chunks:
        print(chunk.text)
# with multiproc, n_process=-1: 53s without 84s

  0%|          | 0/6049 [00:00<?, ?it/s]

me
I
a new social network
Instagram
the ring
my couple days
usage
I
one thing
lack
speaker indication
rooms
a healthy amount
people
the stage
you
yourself
a game
hide
you
who
avatar
the ring
my 2 possible suggestions
wishes
they
Speakers
a “Moderators” section
the top
the room
a “Stage” section
it
it
speaker notifications
_
_
_
a drop down notification
a moderator
the amount
scrolling
who
stage participants
the audience
a moderator
the profile sizes
stage
the number
more avatars
the screen
someone
we
who
it
This app
January
the app
people
stages
mods
the audience
the room
people
others
popular rooms
they
people
profiles
this app
the person
everyone
the person
who
the reporters
I
no need
the African American man
the apps
picture
it
a Caucasian person
We
Chinese Hispanic Dominican
all other races
the United States
you
you guys
the one Caucasian man
who
life
Club house
thousands
others
life
the app
People
this app
I
you guys
the app
It
everyone
rooms
we
shadow
the oddest reasons
it
way to

It
me
I
I
a close friend
it
me
this information
every profile
It
me
a major privacy issue
no possible way
this app
people
my phone number
my profile
all this personal information
I
no way
the app
nothing
I
It
me
a panic attack
I
I
this app
I
It
they
it
our point
reference
contacts
Plenty
powerful keynotes
quality material
the incredible minds
who
their hearts
experiences
motivational energy
Exactly the kind
platform
I
you
I
everyone
I
this app
I
barriers
other social media streams
the dangers
we
social media
My heart
thanks
the developers
a quality media stream
us
My body
heart
mind
soul
ease
Beautiful minds
I
this app
It
such a great and eclectic mix
people
(especially POC
a breadth
backgrounds
the conversations
I
all hours
I
the range
topics
It
my favorite social platform
it
I
the other reviews
It
topics
I
sign
what
conversations
an emoji
appreciation
I
it
it
it
the conversations
point
Twitter
I
something
conversations
nobody
something
they
You
lights
cameras
you
yourself
you
good co

I
a conversation
a silly title
an actress
comedian
I
it
I
I
ages
it
just her snappy retorts
others
the room
Some actors
just everyday people
it
me
I
the randomness
freshness
I
meditations
other conversations
Only one draw
I
only 2 referrals/invites
it
hard picking
who
I
an invite
I
I
them
people
I
Clubhouse
Clubhouse
It
a never-ending infomercial
your future self
the only price
the amount
time
you
the app
the effort
you
it
one
the internet
the level
depth
this app
terms
immediate feedback
other users
you
the right “heart posture
they
this portal
incredibly beautiful and vast worlds
amazing communities
you
This app
an extremely large user base
the coming months
years
more people
it
I
this app
Ipad
somebody
who
a bigger screen
a moderator
speaker
I
a larger screen
a bigger visual
who
who
Nothing
just good conversations
what
I
The invite
the random people
I
I
people
who
they
that pressure
them
someone
who
the platform
I
people
the app
ignorance
arguments
it
a safe community
people
knowled

a much more fluid way
I
we
a messenger
room chat
something
us
at least the moderator


Update
I
Clubhouse
polls
us
a question mark
the room
our picture
order
It
such a great app
it
politics
Certain groups
individuals
their clubhouse accounts
individuals
who
I
it
a glitch
I
this app
a certain group
people
I
a big deal
the discrimination
I
the developers
I
the hands
the owners
this app
the solid truth
I
so many connections
my business
a healthy way
relationships
you
CH
the benefits
your friend and/or family member
who
the waiting list
your phone
the app
I
you
trust
it
your life
a currently speaking icon
the last 3-4 speakers
the top
who
the stage
people
tho
the speaker
a bright red
gold
gold
moderators
others
this room
I
this app
I
2 video
I
different ways
the word
I
my contacts
I
we
a limit
three invites
I
it
topics
my room
I
its hading
itself
their growth / reach
the size
their team
non-spammy or irrelevant rooms
a while
you
your community
it
Great idea
good execution
low hanging fruit

...

you guys
the “my list
I
a show
movie
my list
it
the very bottom
you
something
you
your “my list
the very top
it
a recently added item
you
us
the option
the way
it
     thanks
I
the selections
the movies
everything
one thing
I
you
what language
The description
everything
you
it
it
another language
You
the language
we
this series
Violetta
I
it
it
ya’ll
it
I
ya’ll
it
I
Netflix
some time
a problem
I
Friends
I
the part
Chandler
Monica
those episodes
season
6 episode
it
episode
season
repeats episodes
I
purpose
accident
I
something
it
I
Netflix
a while
the app
I
a couple
suggestions
I
the subtitles
It
the show/movie
I
you
10 seconds
it
the screen
you
the pause button
those two complaints
Netflix
the good work
I
this app
I
the star ratings
I
time
I
it
I
my favorites
I
the app
Netflix
a ten second forward button
it
this app
I
I
a day
something
Netflix
the bad days
Thanks
the consideration
🏼
🏼
just about as good an app
I
It
the navigation
the video player interface
it
Netflix content
This lates

the brief movie descriptions
I
the tags
I
them
they
addition
the descriptions
replacements
them
the star rating system
the current thumbs up/down option
user sharing experiences
Netflix
downloading items
the app
it
a problem
it
me
certain anime episodes
stuff
I
those shows
movies
anime
a update
Inuyasha
other anime
what
I
problems
it
me
it
the layout
the algorithm
recommendations
different devices
the iOS devices
macOS
The only thing
me
you
a series
you
the current season
it
the beginning
this app
I
my Netflix subscription
The iPad app
the latest update
Netflix
a miserable experience
It
trailers
me
I
a show
I
full screen
the inset screen
email
another app
Netflix
the Netflix app
trailers
me
I
full screen
I
the show
I
the audio
the trailer
the show
you
an app
Love Netflix
the last update
the “go back 10 seconds” button
I
a line
I
a little context
I
I
I
a reason
this feature
I
it
I
something
you
I
it
I
you
more seasons
me
I
Netflix
the seasons
a show
The show
Henry danger
another Bella
t

my family member’s profile
I
her recommendations
the things
I
I
I
shows
I
attention
what episode
I
I
a profile
mine
I
the guess work
I
Netflix
about 4 years
it
's
the fact
some good shows
movies
4 and 5 star ratings
we
a limit
we
our list
it
a lot
Y'all
you
1 less user
the hell
adult swim shows
a request
a review
I
you guys
all the seasons
episodes
every show
you guys
I
the shows
the episodes
your app
more people
it
what
you
us
A REQUEST
A DEMAND
I
the kind
person
other people
I
I
Netflix
They
good shows
it
me
updates
My whole issue
the app
touch 6th generation
its iOS
something
that extent
I
Netflix
they
hey apple devices
IOS
the devices
it
Great service
my opinion
me
I
you
greys anatomy
I
it
it
I
you
you
pretty little liars
Greys anatomy
pretty little liars
amazing shows
you
them
I
you
a high rating
I
you
your decisions
I
this app
I
anime
I
attack
Titans
one season
4 seasons
like haikyuu
2 seasons
my point
the full seasons
they
all the seasons
the anime episodes
seasons
i
I
a anime
a

their video
freeze
ios 14 iphone
really big issues
notification
we
any updates
who
it
what rating showd
the app
good resolve
the prblm
very big problem
it
your giving update
it
waste
I
the app
its really a great alternative
exchange
tik tok
some immediate attention
these problems
the app
its demand
The app
notifications
the app
followers
other users
we
the same data
ourselves
this issue
Moj
the best ♥️
tiktok
I
it
Moj
me
good recognition
support
Moj team
excellent ♥️i
a request
I
the duet option
delete option
you
I
an iPhone XR
I
this app
I
notifications
I
I
the videos
I
I
the option
it
only one problem
this app
we
gallery
Moj
it
the same post
we
it
the best app
🏻
I
Singh
My user name
I
new user
I
my first and new video
Video quality
I
this application
I
new video content
daily bases
I
my videos
me
i
a video
it
the same video
you
other popular persons
tik tok
small users
me
my moj user
I
Sahil_zehen
I
iPad iOS
MOJ app
27 files
it
button
some photos
two videos
I
my gallery
video
MOJ app

the problem
They
not even clear videos
they
100k likes
I
tiktok
I
lots
fans
my videos
my account
I
UAE
i
app
it
it
a lot
issues
the app
me
any portal
the issues
Download option
iPhone
the matter
I
full enjoy
this app
💝💝
I
any notification
🧐
Thanq
I
me
Very good app
it
duet
share
Zoom Option
Instagram
Tejas kayande
New punjabi songs
My mobile
sound
I
iPhone 11 Pro Max
I
those videos
I
this bug
These all a drama
North indians
This app
TikTok
Worst UI
unwanted app
They
all your private photos
videos
it
your safety
Some effects
tik tok
sparkle
hearts
pls
App
I
a video
I
any video
option
sounds
videos
duet option
Best app
creators
Moj kya MA
Kara la
my moj
follower
one video view
Application
I
other users followers
videos
gallery
black screen
the hastag
It
awesome app
this entertaining app
it
I
my
favourite videos
This experience
seriously plzz guys
it
tik tok

Tiktok
amazing app
this app
we
even a one percent rating
it
error
a video
My phone
iPhone
this app
it
type
a big fan
tiktok
this ap

## Topic Modeling with Textacy via Scikit-Learn

In [19]:
tokenized_docs = ((term.lemma_ for term in textacy.extract.terms(doc, ngs=1, ents=True)) for doc in corpus)

In [21]:
from textacy.representations.vectorizers import Vectorizer
vectorizer = Vectorizer(tf_type="linear", idf_type="smooth", norm="l2",min_df=3, max_df=0.95)

In [22]:
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)
doc_term_matrix

<6049x4739 sparse matrix of type '<class 'numpy.float64'>'
	with 156648 stored elements in Compressed Sparse Row format>

In [28]:
from textacy.tm import TopicModel

In [29]:
model = TopicModel("nmf", n_topics=20)

In [30]:
model.fit(doc_term_matrix)

/home/nirant/miniconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "


In [31]:
doc_topic_matrix = model.transform(doc_term_matrix)

In [35]:
for topic_idx, top_terms in model.top_topic_terms(vectorizer.id_to_term, topics=range(20)):
    print("topic", topic_idx, ":", "   ".join(top_terms))

topic 0 : watch   episode   list   continue   want   thing   update   time   screen   fix
topic 1 : driver   Uber   ride   time   charge   cancel   uber   minute   $   pick
topic 2 : app   use   nice   work   update   fix   issue   open   problem   let
topic 3 : invite   send   clubhouse   need   friend   use   got   get   reserve   username
topic 4 : love   absolutely   day   ️   far   idea   conversation   thing   network   concept
topic 5 : account   delete   ready   email   create   sign   log   password   let   card
topic 6 : good   far   App   thing   platform   job   hand   🏻   take   🔥
topic 7 : people   room   social   platform   connect   medium   Clubhouse   learn   new   way
topic 8 : season   anime   2   come   Netflix   episode   3   4   series   watch
topic 9 : great   idea   network   networking   platform   connect   way   learn   experience   potential
topic 10 : like   feel   cool   thing   need   nice   kid   people   guy   minded
topic 11 : amazing   App   truly   

In [37]:
for topic_idx, top_docs in model.top_topic_docs(doc_topic_matrix, topics=[0,1], top_n=2):
    print(topic_idx)
    for j in top_docs:
        print(j, corpus[j])
        print()

0
3704 Don’t get me wrong, I love Netflix and its variety with show and movies. One thing I have been having problems with is needing to watch the shows I don’t like . Let me explain. If I watch like the first episode of a certain show and I decide I don’t want to watch it, I can’t take it out of my continue watching list. Then, I would have to skip through every episode until the credits come on and the buttons come up saying watch next episode. Sometimes even that doesn’t work. I have three shows on my continue watching list that I don’t want to watch but can’t get it off of the list. It would be very nice if there was a button like the add to list button but it removes the show or movie from the continue watching list. I just thought I might bring this up because this annoys me.

4609 Netflix is great, but when I start a show and I don’t want to watch it I’m still supposed to finish it or it will be on my to continue watching list which I don’t like it to be on. So, what I’m really 